In [1]:
import numpy

import pycuda
import pycuda.autoinit
import pycuda.compiler
import pycuda.driver
import pycuda.gpuarray

In [50]:
module = pycuda.compiler.SourceModule("""
__global__ void reduce_a(int length, int* parents, float* mutablescan) {
    unsigned int i = threadIdx.x + blockIdx.x*blockDim.x;
    if (i < length) {
        for (int d = 1;  d < length;  d *= 2) {
            if (i >= d  &&  parents[i] == parents[i - d]) {
                mutablescan[i] = mutablescan[i] + mutablescan[i - d];
            }
            __syncthreads();
        }
    }
}

__global__ void reduce_b(int length, int* starts, int* stops, float* scan, float* output) {
    unsigned int i = threadIdx.x + blockIdx.x*blockDim.x;
    if (i < length) {
        if (starts[i] == stops[i]) {
            output[i] = 0.0;
        }
        else {
            output[i] = scan[stops[i] - 1];
        }
    }
}
""")

reduce_a = module.get_function("reduce_a")
reduce_b = module.get_function("reduce_b")

In [51]:
# counts to offsets
counts = numpy.random.poisson(5, 2000)
offsets = numpy.empty(len(counts) + 1, dtype=numpy.int32)
offsets[0] = 0
numpy.cumsum(counts, out=offsets[1:])

# content
content = numpy.random.normal(1, 0.00001, offsets[-1]).astype(numpy.float32)

# parents
parents = numpy.zeros(len(content), dtype=numpy.int32)
numpy.add.at(parents, offsets[offsets != offsets[-1]][1:], 1)
numpy.cumsum(parents, out=parents)

# mutable contents on GPU
scan = pycuda.gpuarray.to_gpu(content)

In [52]:
# run!
reduce_a(numpy.int32(len(parents)), pycuda.driver.In(parents), scan,
         block=(1024, 1, 1), grid=(len(content) // 1024 + 1, 1))